<a href="https://colab.research.google.com/github/christianwarmuth/transformer_adapter_bias_evaluation/blob/main/Adapter_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Evaluating Adapter 

In [10]:
!pip install adapter-transformers
!pip install datasets
!pip install dataloader

In [11]:
import pandas as pd

import dataloader
from collections import defaultdict 

# Dataset Loading - Stereoset

In [13]:
stereoset = dataloader.StereoSet("/content/dev.json")

In [14]:
stereoset

In [15]:
intersentence_examples = stereoset.get_intersentence_examples() 
intrasentence_examples = stereoset.get_intrasentence_examples()

In [26]:
intrasentence_examples[1].sentences[1].gold_label

'anti-stereotype'

In [28]:
id2term = {}
id2gold = {}
id2score = {}
example2sent = {}
domain2example = {"intersentence": defaultdict(lambda: []), "intrasentence": defaultdict(lambda: [])}

for example in intrasentence_examples:
  for sentence in example.sentences:
    id2term[sentence.ID] = example.target
    id2gold[sentence.ID] = sentence.gold_label
    example2sent[(example.ID, sentence.gold_label)] = sentence.ID
    domain2example['intrasentence'][example.bias_type].append(example)

for example in intersentence_examples:
  for sentence in example.sentences:
    id2term[sentence.ID] = example.target
    id2gold[sentence.ID] = sentence.gold_label
    example2sent[(example.ID, sentence.gold_label)] = sentence.ID
    domain2example['intersentence'][example.bias_type].append(example)

In [34]:
# When predictions are already done
for sent in self.predictions.get('intrasentence', []) + self.predictions.get('intersentence', []):
            self.id2score[sent['id']] = sent['score']

{('fdc01ba6042b41f0c894cdccc149f5b4',
  'anti-stereotype'): '107a3b2e248a218017cf1ba6a22f2c76',
 ('fdc01ba6042b41f0c894cdccc149f5b4',
  'unrelated'): '8abf83f356f84d55b3d823b255c47fbf',
 ('fdc01ba6042b41f0c894cdccc149f5b4',
  'stereotype'): '730a20c60dbc546b9643333ffa11338b',
 ('855484664e331c670e1bbca3ec708d1c',
  'stereotype'): 'da3ffe0b63e23ecead967407f350c1b9',
 ('855484664e331c670e1bbca3ec708d1c',
  'anti-stereotype'): 'b45db81e8d5ac4a1ca21b8c7af1b1ef8',
 ('855484664e331c670e1bbca3ec708d1c',
  'unrelated'): 'ab8bccba80cee897ce52924a539ed9cb',
 ('7827e707f511b675981d6bbb1c3860ba',
  'unrelated'): '9e736b29e650f6945a3b9a4500354e11',
 ('7827e707f511b675981d6bbb1c3860ba',
  'anti-stereotype'): 'bfc67efc21a48f2cd9a848fa3b792ca6',
 ('7827e707f511b675981d6bbb1c3860ba',
  'stereotype'): '2c4608682eff3ee1b81de7a13facc8ae',
 ('919d0ea5874335e43d162b917703a475',
  'unrelated'): 'd538c7dc90f0956f3348befdff085c1b',
 ('919d0ea5874335e43d162b917703a475',
  'stereotype'): '12f89da3bbf7d69a0bdff53

In [ ]:
domain2example

In [36]:
!pip install colorama

In [83]:
from eval_template import *
import dataloader

class Args:
  no_cuda = False
  input_file = '/content/dev.json'
  output_file = '/content/predictions.json'

args=Args()
#class input_args:
#  def __init__(self, no_cuda, input_file, output_file, max_seq_length):
#    self.no_cuda = no_cuda
 #   self.input_file = input_file
 #   self.output_file = output_file
#    self.max_seq_length = max_seq_length

#args = input_args(False, "/content/dev.json", "/content/predictions.json", 128)
#args = {#
#    "no_cuda": False, 
#    "input_file": "/content/dev.json", 
#    "output_file":"/content/predictions.json"
#}

print(vars(args))

evaluator = BiasEvaluator(**vars(args))
results = evaluator.evaluate()

#if args.output_file is not None:
#  output_file = args.output_file
#else:
output_file = "/content/predictions.json"

output_file = os.path.join(args.output_dir, output_file)
with open(output_file, "w+") as f:
  json.dump(results, f, indent=2)


{}
Loading data/bias.json...


FileNotFoundError: ignored

In [75]:
class Args:
  no_cuda = False
  input_file = '/content/sample_data/dev.json'
  output_file = '/content/sample_data/predictions.json'

args=Args()

In [67]:
print(vars(args))

{}
